In [2]:
import numpy as np

In [3]:
#Data load
test_data_filepath = '/Users/yasminebenmessaoud/Library/CloudStorage/OneDrive-ITU/Machine Learning/JustTwo/fashion_test.npy'
train_data_filepath = '/Users/yasminebenmessaoud/Library/CloudStorage/OneDrive-ITU/Machine Learning/JustTwo/fashion_train.npy'

# Load the files
fashion_train = np.load(train_data_filepath)
fashion_test = np.load(test_data_filepath)

In [4]:
class NeuralNet(object):
    RNG = np.random.default_rng()


    def __init__(self, topology:list[int] = []):
        self.topology    = topology
        self.weight_mats = []

        self._init_matrices()


    def _init_matrices(self):
        #-- set up matrices
        if len(self.topology) > 1:
            j = 1
            for i in range(len(self.topology)-1):
                num_rows = self.topology[i]
                num_cols = self.topology[j]

                mat = self.RNG.random(size=(num_rows, num_cols))
                self.weight_mats.append(mat)

                j += 1
                
    
    def feedforward(self, input_vector):
        I = input_vector
        
        for idx, W in enumerate(self.weight_mats):
            
            I = np.dot(I, W)
            
            if idx == len(self.weight_mats) - 1:
                out_vector = np.tanh(I)  #output layer
            else:
                I          = np.tanh(I)  #hidden layers
            
        return out_vector

In [5]:
nnet = NeuralNet(topology=[3,4,2])

In [6]:
nnet.feedforward([0.3, 5, 1])

array([0.91323492, 0.96956638])